# Homework 3 - What movie to watch tonight?

## Group 14:
+ Riccardo Ceccaroni,
+ Angelo  Berardi,  
+ Yifei Wei.  


## 1. Data collection


### 1.1. Get the list of movies


The collector_utils file contains two functions:
+ **data_html_to_dataframe**: that reads the links in the # .html movie files and returns a dataframe with all the links of the wikipedia pages;


In [ ]:
def data_html_to_dataframe(file_name):
    soup = bs(open(file_name), "html.parser")
    wiki_links = {'idx': [],
                  'links': []}

    for tr in soup.findAll("tr"):
        for td in tr.findAll("td"):
            if 'http' in td.text:
                wiki_links['links'].append(td.text)
            else:
                wiki_links['idx'].append(td.text)
    df = pd.DataFrame(wiki_links, columns=['idx', 'links'])

    return df

+ **save_webpage**: that saves wikipedia pages in html in the Wikipedia /

In [ ]:
def save_webpage(url, file_name):
    html = urlopen(url)
    page_content = html.read()
    with open('Wikipedia/' + file_name, 'wb') as fid:
        fid.write(page_content)

Importing the library collector runs a script that requires the file number and the index from which to start.
Files are saved in the Wikipedia /

In [ ]:
import collector

### 1.2. Crawl Wikipedia

The parser_utils.py file contains:
+ the function to clean up the text:

In [ ]:
def dividi_parole(string):
    for i in range(1, len(string) - 1):
        if string[i - 1].islower() and string[i].isupper():
            string = string[0:i] + ", " + string[i:len(string)]
        elif string[i - 1] == '.' and string[i].isupper():
            string = string[0:i] + ", " + string[i:len(string)]
    return string

+ the parser, that collects information from the pages in the Wikipedia / directory:

In [ ]:
def html_to_dict(file_name):
    film_dict = {}
    with open(file_name, 'rb') as html:
        soup = bs(html, 'html.parser')

    if 'This disambiguation page' in soup.text:
        pass  # this is a disambiguation page, not film page
    else:
        # Title Page
        page_title = soup.title.text
        page_title = re.sub(r' - Wikipedia', '', page_title)
        film_dict.update({"Title": page_title})

        # Paragraphers (Intro and Plot)
        p = soup.find_all("p")

        if len(p) >= 1:
            intro = p[0].text
            intro = re.sub(r'\n', '', intro)
            intro = re.sub(r'\[\d*\]', '', intro)
            film_dict.update({"Intro": intro})
        if len(p) >= 2:
            plot = p[1].text
            plot = re.sub(r'\n', '', plot)
            plot = re.sub(r'\[\d*\]', '', plot)
            film_dict.update({"Plot": plot})

        # URL
        urls = soup.find("link", {"rel": "canonical", 'href': True})
        film_dict.update({"Wikipedia Url": urls.get('href')})

        # Film Infobox
        infobox = soup.find_all("table", {"class": "infobox vevent"})

        if infobox:
            infobox = infobox[0]

            tr = infobox.find_all('tr')

            for i in range(len(tr)):
                tr[i] = re.sub(r'\[\d*\]', '', tr[i].text)
                tr[i] = tr[i].replace("  ", ", ")
                tr[i] = tr[i].replace("\n", ", ")
                tr[i] = unicodedata.normalize("NFKD", tr[i])

                if 'Directed by' in tr[i][:11]:
                    film_dict.update({"Directed by": tr[i][11:].strip(', ')})
                elif 'Produced by' in tr[i][:11]:
                    film_dict.update({"Produced by": dividi_parole(tr[i][11:].strip(', '))})
                elif 'Written by' in tr[i][:10]:
                    film_dict.update({"Written by": dividi_parole(tr[i][10:].strip(', '))})
                elif 'Starring' in tr[i][:8]:
                    film_dict.update({"Starring": dividi_parole(tr[i][8:].strip(', '))})
                elif 'Narrated by' in tr[i][:11]:
                    film_dict.update({"Narrated by": dividi_parole(tr[i][11:].strip(', '))})
                elif 'Music by' in tr[i][:8]:
                    film_dict.update({"Music by": dividi_parole(tr[i][8:].strip(', '))})
                elif 'Screenplay by' in tr[i][:13]:
                    film_dict.update({"Screenplay by": dividi_parole(tr[i][13:].strip(', '))})
                elif 'Story by' in tr[i][:8]:
                    film_dict.update({"Story by": dividi_parole(tr[i][8:].strip(', '))})
                elif 'Cinematography' in tr[i][:14]:
                    film_dict.update({"Cinematography": dividi_parole(tr[i][14:].strip(', '))})
                elif 'Edited by' in tr[i][:9]:
                    film_dict.update({"Edited by": dividi_parole(tr[i][9:].strip(', '))})
                elif 'Productioncompany' in tr[i][:17]:
                    film_dict.update({"Production Company": dividi_parole(tr[i][17:].strip(', '))})
                elif 'Distributed by' in tr[i][:14]:
                    film_dict.update({"Distributed by": dividi_parole(tr[i][14:].strip(', '))})
                elif 'Release date' in tr[i][:12]:
                    film_dict.update({"Release date": dividi_parole(tr[i][12:].strip(', '))})
                elif 'Running time' in tr[i][:12]:
                    film_dict.update({"Running time": dividi_parole(tr[i][12:].strip(', '))})
                elif 'Country' in tr[i][:7]:
                    film_dict.update({"Country": dividi_parole(tr[i][7:].strip(', '))})
                elif 'Language' in tr[i][:8]:
                    film_dict.update({"Language": dividi_parole(tr[i][8:].strip(', '))})
                elif 'Budget' in tr[i][:6]:
                    film_dict.update({"Budget": tr[i][6:].strip(', ')})
                elif 'Box office' in tr[i][:10]:
                    film_dict.update({"Box office": tr[i][10:].strip(', ')})
                else:
                    pass
    return film_dict

+ function to save all the information in a tsv file

In [ ]:
def save_tsv_dataframe(list_info_film):
    dataframe = pd.DataFrame(list_info_film)
    n_file = input("Number of movies file (1, 2, 3): ")
    name_file = 'Wikipedia/tsv/movie' + n_file + '.tsv'
    dataframe.to_csv(name_file, sep='\t', index=False)

Importing the parser library, the parser function is launched on all the files in the Wikipedia/ directory and the user is asked to save the file:

In [ ]:
import parser

### 2. Search Engine

The index_utils.py file contains all the functions to clean up the text, for example:

In [ ]:
def format_text(text):
    text = text.lower()
    text = tokenizer(text)
    text = remove_stop_word(text)
    text = word_lemmatizer(text)
    return " ".join(text)

 and create a vocabulary:

In [ ]:
def generate_vocabulary_df(dataframe_df):
    vocabulary = list(set((' '.join(dataframe_df['Intro+Plot'].tolist())).split(' ')))
    vocabulary.remove('')
    vocabulary = pd.DataFrame(vocabulary, columns=['Word'])
    vocabulary.to_json(r'Json\vocabulary.json', index=False, orient='table')
    return vocabulary

The index.py file contains the class that define an index, with the function for search file and create scores:

In [ ]:
class InvertedIndex:
    """
    Inverted Index class.
    """

    def __init__(self, index):
        self.index = index

    def __repr__(self):
        """
        String representation of the Database object
        """
        return str(self.index)

    def index_document(self, document):
        """
        Process a given document, save it to the DB and update the index.
        """

        # Remove punctuation from the text.
        clean_text = index_utils.format_text(document['text'])
        terms = clean_text.split(' ')
        appearances_dict = dict()
        # Dictionary with each term and the frequency it appears in the text.
        for term in terms:
            term_frequency = appearances_dict[term]['frequency'] if term in appearances_dict else 0
            appearances_dict[term] = {'docId': document['id'], 'frequency': (term_frequency + 1)}

        # Update the inverted index
        update_dict = {key: [appearance]
                       if key not in self.index
                       else self.index[key] + [appearance]
                       for (key, appearance) in appearances_dict.items()}
        self.index.update(update_dict)
        # Add the document into the database
        return document

    def lookup_conjunctive_query(self, query):
        try:
            with open(r'Json\inverted_index_dict.json', 'r') as f:
                self.index = json.load(f)
        except:
            print("Load Error Database")
        r = {}
        for term in query.split(' '):
            if term in self.index:
                r.update({term: self.index[term]})
            else:
                r.update({term: []})
        j = 0
        l = []
        for d in r:
            t = []
            for i in range(len(r[d])):
                t.append(r[d][i]['docId'])
            l.append(set(t))
            j += 1
        idx_set = l[0]
        for i in range(1, len(l)):
            idx_set = idx_set.intersection(l[i])

        return dataframe[['Title', 'Intro', 'Wikipedia Url']].iloc[list(idx_set)].reset_index(drop=True)

    def generate_tfidf(self):
        D = len(dataframe_df[['Intro+Plot']])
        for word in self.index:
            d = len(self.index[word])
            for doc_dict in self.index[word]:
                tfidf = (doc_dict['frequency'] / len(dataframe_df['Intro+Plot'].iloc[doc_dict['docId']].split(" "))) * (1 + log(D/d))
                doc_dict['tfidf'] = tfidf
        with open(r'Json\inverted_index_tfidf.json', 'w') as f:
            json.dump(self.index, f)

    def lookup_conjunctive_query_and_ranking_score(self, query):
        # import inverted index tfidf database
        try:
            with open(r'Json\inverted_index_tfidf.json', 'r') as f:
                self.index = json.load(f)
        except:
            with open(r'Json\inverted_index_dict.json', 'r') as f:
                self.index = json.load(f)
            self.generate_tfidf()
            with open(r'Json\inverted_index_tfidf.json', 'r') as f:
                self.index = json.load(f)

        index_for_query = {term: self.index[term] for term in query.split(' ') if term in self.index}

        # calculate tfidf for query
        query_tfidf = []
        query_split = query.split(' ')
        tf = 1 / len(query_split)
        for word in query_split:
            try:
                idf = 1 + log(len(dataframe_df[['Intro+Plot']])/len(self.index[word]))
            except:
                idf = 1
            query_tfidf.append({'word': word, 'tfidf': tf*idf})

        # calculate cosine similarity
        ## index -> dataframe
        index_for_query_df = {word: {doc_dict['docId']: doc_dict['tfidf'] for doc_dict in index_for_query[word]}
                              for word in index_for_query}
        index_for_query_df = pd.DataFrame(index_for_query_df).fillna(0)

        ## Cosine similarity
        col_list = list(index_for_query_df)

        index_for_query_df.loc[:, 'den'] = ((index_for_query_df.loc[:, col_list] ** 2)[col_list].sum(axis=1)) ** (1/2)
        index_for_query_df.loc[:, 'num'] = 0
        query_mod = 0

        for i in range(len(query_tfidf)):
            if query_tfidf[i]['word'] in index_for_query_df:
                index_for_query_df.loc[:, 'num'] += index_for_query_df.loc[:, query_tfidf[i]['word']] * query_tfidf[i]['tfidf']
            query_mod += (query_tfidf[i]['tfidf'] ** 2)
        query_mod = sqrt(query_mod)
        index_for_query_df.loc[:, 'den'] *= query_mod
        index_for_query_df.loc[:, 'Similarity'] = index_for_query_df.loc[:, 'num'] / index_for_query_df.loc[:, 'den']
        df_similarity = pd.merge(dataframe[['Title', 'Intro', 'Wikipedia Url']], index_for_query_df[['Similarity']],
                                 left_index=True, right_index=True, sort=True)
        df_similarity = df_similarity.sort_values(by=['Similarity'], ascending=False)

        return df_similarity[df_similarity['Similarity'] >= 0.75].reset_index(drop=True)

    def save_inverted_index(self):
        with open(r'Json\inverted_index_dict.json', 'w') as f:
            json.dump(self.index, f)


The function **generate_tfidf** generates the tf index and idf index:
<p>The <b>TF</b> (term frequency) is defined as:</p> 

<p>$TF = n_w/N_w$</p>

<p>where $n_w$ is the number of occurences of the word in document, and $N_w$ the total number of words in the document.</p>

<p>while the <b>IDF</b> (inverse document frequency) is:</p>

<p>$IDF = 1 + log_{10}\left(\frac{N_d}{n_d}\right)$</p>

<p>where $N_d$ is the total number of documents and $n_d$ the number of documents containing the word</p>

In the main.py file an object InvertedIndex is created and with this the requests for research are managed:

In [1]:
import utils
import index
import index_utils


inverted_index = index.InvertedIndex(index.idx)

For example:

### 2.1. Conjunctive query

In [2]:
search_term = input("Enter term(s) to search: ")
inverted_index.lookup_conjunctive_query(index_utils.format_text(search_term))

Enter term(s) to search:  Disney Movies


,Title,Intro,Wikipedia Url
0,Under Wraps (film),Under Wraps is a 1997 television film directed...,https://en.wikipedia.org/wiki/Under_Wraps_(film)
1,Annie (1982 film),Annie is a 1982 American musical comedy-drama ...,https://en.wikipedia.org/wiki/Annie_(1982_film)
2,Summer Magic,Summer Magic is a 1963 Walt Disney Productions...,https://en.wikipedia.org/wiki/Summer_Magic
3,The Last Starfighter,The Last Starfighter is a 1984 American space ...,https://en.wikipedia.org/wiki/The_Last_Starfig...
4,The Cat Concerto,The Cat Concerto is a 1947 American one-reel a...,https://en.wikipedia.org/wiki/The_Cat_Concerto
5,Machine Gun Mama,Machine Gun Mama is a 1944 American musical co...,https://en.wikipedia.org/wiki/Machine_Gun_Mama
6,Rango (2011 film),Rango is a 2011 American computer-animated wes...,https://en.wikipedia.org/wiki/Rango_(2011_film)
7,Northern Lights (1997 film),Northern Lights is a 1997 television film base...,https://en.wikipedia.org/wiki/Northern_Lights_...
8,WALL-E,WALL-E (stylized with an interpunct as WALL·E)...,https://en.wikipedia.org/wiki/WALL-E
9,The Rugrats Movie,The Rugrats Movie is a 1998 American animated ...,https://en.wikipedia.org/wiki/The_Rugrats_Movie


### 2.2. Conjunctive query & Ranking score

In [3]:
search_term = input("Enter term(s) to search: ")
inverted_index.lookup_conjunctive_query_and_ranking_score(index_utils.format_text(search_term))

Enter term(s) to search:  Disney Movies


,Title,Intro,Wikipedia Url,Similarity
0,The Rugrats Movie,The Rugrats Movie is a 1998 American animated ...,https://en.wikipedia.org/wiki/The_Rugrats_Movie,1.000000
1,WALL-E,WALL-E (stylized with an interpunct as WALL·E)...,https://en.wikipedia.org/wiki/WALL-E,1.000000
2,The Last Starfighter,The Last Starfighter is a 1984 American space ...,https://en.wikipedia.org/wiki/The_Last_Starfig...,1.000000
3,Annie (1982 film),Annie is a 1982 American musical comedy-drama ...,https://en.wikipedia.org/wiki/Annie_(1982_film),1.000000
4,The Avengers (2012 film),Marvel's The Avengers (classified under the na...,https://en.wikipedia.org/wiki/The_Avengers_(20...,1.000000
5,The Cat Concerto,The Cat Concerto is a 1947 American one-reel a...,https://en.wikipedia.org/wiki/The_Cat_Concerto,1.000000
6,She (1935 film),She is a 1935 American film produced by Merian...,https://en.wikipedia.org/wiki/She_(1935_film),1.000000
7,Rango (2011 film),Rango is a 2011 American computer-animated wes...,https://en.wikipedia.org/wiki/Rango_(2011_film),1.000000
8,Machine Gun Mama,Machine Gun Mama is a 1944 American musical co...,https://en.wikipedia.org/wiki/Machine_Gun_Mama,1.000000
9,Sullivan's Travels,Sullivan's Travels is a 1941 American comedy f...,https://en.wikipedia.org/wiki/Sullivan%27s_Tra...,1.000000


### 3. Define a new score!

We defined a new score with this algorithm:

**Jaccard similitude:** The Jaccard index, also known as Intersection over Union and the Jaccard similarity coefficient (originally given the French name coefficient de communauté by Paul Jaccard), is a statistic used for gauging the similarity and diversity of sample sets. The Jaccard coefficient measures similarity between finite sample sets, and is defined as the size of the intersection divided by the size of the union of the sample sets:

$J(A,B) = {{|A \cap B|}\over{|A \cup B|}} = {{|A \cap B|}\over{|A| + |B| - |A \cap B|}}$

In [ ]:
def get_jaccard_sim(str1, str2):
    a = set(str1.split())
    b = set(str2.split())
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))

In the index.py file there is the function define_new_score(query, k) that take as input 'query', the subject to search, and k as biggest number all the results of the research. The function search by 'Title', 'Written by', 'Directed by', 'Distributed by'.

In [ ]:
def define_new_score(query, k):
    # Trovare per cosa cercare + ripulire testo
    heap = []
    try:
        what = pd.read_json(r'Json\dataframe_format_all.json', orient='table')
    except:
        what = index_utils.generate_format_for_new_score(dataframe)

    what = what[['All']]
    format_query = index_utils.format_text(query)

    for i in range(len(what)):
        heapq.heappush(heap, [index_utils.get_jaccard_sim(format_query, str(what.iloc[i])), i])
    result = {int(x[1]): float(x[0])for x in heapq.nlargest(k, heap) if float(x[0]) > 0}

    if result:
        new_score_df = pd.DataFrame(heapq.nlargest(k, heap), columns=['Similarity', 'idx'])
        df_similarity = pd.merge(dataframe[['Title', 'Intro', 'Wikipedia Url']], new_score_df[['Similarity', 'idx']],
                                 left_index=True, right_on='idx', sort=True)
        df_similarity = df_similarity.sort_values(by=['Similarity'], ascending=False)
        return df_similarity[['Title', 'Intro', 'Wikipedia Url', 'Similarity']]
    else:
        return 'Not Found'

In [4]:
import utils
import index
import index_utils

In [5]:
search_term = input("Enter term(s) to search: ")
result, actor = index.define_new_score(search_term, 10)

Enter term(s) to search:  Alice in wonderland


In [6]:
utils.print_actor(actor)

[['Charlotte Henry', ' W. C. Fields', ' Richard Arlen', ' Edna May Oliver', ' Cary Grant', ' Gary Cooper', ' Edward Everett Horton', ' Baby Le', ' Roy'], ['Kathryn Beaumont', ' Ed Wynn', ' Richard Haydn', ' Sterling Holloway', ' Jerry Colonna', ' Verna Felton', " J. Pat O'Malley", ' Bill Thompson', ' Joseph Kearns', ' Dink Trout', ' James Mac', ' Donald'], ['Johnny Depp', ' Anne Hathaway', ' Helena Bonham Carter', ' Crispin Glover', ' Matt Lucas', ' Mia Wasikowska', ' Alan Rickman', ' Stephen Fry', ' Michael Sheen', ' Timothy Spall'], ['May Clark', ' Cecil M. Hepworth', ' Mrs. Cecil Hepworth', ' Norman Whitten'], ['Natalie Gregory', ' Red Buttons', ' Anthony Newley', ' Jayne Meadows', ' Carol Channing', ' Roddy  Dowall', ' Ann Jillian', ' Pat MoritaRobert Morley'], ['Jayaram', ' Vineeth', ' SandhyaLaya'], ['Ellen Burstyn', ' Kris Kristofferson', ' Diane Ladd', ' Jodie Foster', ' Alfred Lutter'], ['Linda Miller', ' Mildred Clinton', ' Paula Sheppard', ' Niles  Master', ' Brooke Shields'

In [8]:
result

,Title,Intro,Wikipedia Url,Similarity
2,Alice in Wonderland (1933 film),Alice in Wonderland is a 1933 American pre-Cod...,https://en.wikipedia.org/wiki/Alice_in_Wonderl...,0.166667
1,Alice in Wonderland (1951 film),Alice in Wonderland is a 1951 American animate...,https://en.wikipedia.org/wiki/Alice_in_Wonderl...,0.166667
0,Alice in Wonderland (2010 film),Alice in Wonderland is a 2010 American dark fa...,https://en.wikipedia.org/wiki/Alice_in_Wonderl...,0.166667
5,Alice in Wonderland (1903 film),Alice in Wonderland is a 1903 British silent f...,https://en.wikipedia.org/wiki/Alice_in_Wonderl...,0.153846
4,Alice in Wonderland (1985 film),Alice in Wonderland is a 1985 two-part made-fo...,https://en.wikipedia.org/wiki/Alice_in_Wonderl...,0.153846
3,Alice in Wonderland (2005 film),Alice in Wonderland is a 2005 Malayalam langua...,https://en.wikipedia.org/wiki/Alice_in_Wonderl...,0.153846
9,Alice Doesn't Live Here Anymore,Alice Doesn't Live Here Anymore is a 1974 Amer...,https://en.wikipedia.org/wiki/Alice_Doesn%27t_...,0.090909
8,"Alice, Sweet Alice","Alice, Sweet Alice (originally titled Communio...","https://en.wikipedia.org/wiki/Alice,_Sweet_Alice",0.090909
7,Silkwood,Silkwood is a 1983 American biographical drama...,https://en.wikipedia.org/wiki/Silkwood,0.090909
6,The Disappearance of Alice Creed,The Disappearance of Alice Creed is a 2009 Br...,https://en.wikipedia.org/wiki/The_Disappearanc...,0.090909


### 4. Algorithmic question
You are given a string, s. Let's define a subsequence as the subset of characters that respects the order we find them in s. For instance, a subsequence of "DATAMINING" is "TMNN". Your goal is to define and implement an algorithm that finds the length of the longest possible subsequence that can be read in the same way forward and backwards. For example, given the string "DATAMININGSAPIENZA" the answer should be 7 (dAtamININgsapIenzA).

In [4]:
def algorithm_palindrome_sub_string_len(string, idx_start, idx_end):
    """
    recursive algorithm
    """

    if idx_end < idx_start:
        return 0
    if idx_end == idx_start:
        return 1
    if string[idx_start] == string[idx_end]:
        return algorithm_palindrome_sub_string_len(string, idx_start+1, idx_end-1) + 2
    right = algorithm_palindrome_sub_string_len(string, idx_start+1, idx_end)
    left = algorithm_palindrome_sub_string_len(string, idx_start, idx_end-1)

    if right > left:
        return right
    else:
        return left


def palindrome_sub_string_len(string):
    return algorithm_palindrome_sub_string_len(string, 0, len(string)-1)


print(palindrome_sub_string_len(input('Insert string: ')))

Insert string:  dataminingsapienza


7
